In [ ]:
#ライブラリをインポート
import os
import re
import csv
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


In [ ]:
#定数を定義
BINS = 10000  #ヒストグラムのビンの数
EPSILON = .00001  #スムージングパラメータ
UPPER_LIMIT = 1.1 #静止区間の上限
LOWER_LIMIT = 0.9 #静止区間の加減
STATIONARY_INTERVALS = 5  #静止区間除去のサンプルの間隔(何サンプル静止区間が連続したら除去するか)

In [ ]:
#データセットのファイル名と周波数を取得する関数
def get_filename_and_Hz(path: str) -> list[int, str]:
    filename = os.listdir(path)
    filename_and_Hz=[]

    for file in filename:
        Hz = re.search(r'\d+', file)
        if Hz:  #数字の入っていないファイル名があるとエラーを吐くので、このif文でチェックする
            filename_and_Hz.append([int(Hz.group(0)), file])

    return filename_and_Hz

In [ ]:
#ファイル名と周波数を分けて出力する関数
def divide_filename_and_Hz(filename_and_Hz: list[int, str]) -> tuple[list[int], list[str]]:
    Hz = []
    filename = []
    for row in filename_and_Hz:
      Hz.append(row[0])
      filename.append(row[1])

    return Hz, filename

In [ ]:
#加速度データのCSVファイルから3軸加速度を取得する関数
def get_acceleration(filename: str) -> tuple[list[float], list[float], list[float]]:
    AccX, AccY, AccZ = [], [], []
    #ｘ軸,y軸,z軸の値を配列に格納
    with open(filename) as f:
        reader = csv.reader(f)
        for row in reader:
            AccX.append(float(row[2]))
            AccY.append(float(row[3]))
            AccZ.append(float(row[4]))

    return AccX, AccY, AccZ

In [ ]:
#静止区間を除去する関数
def remove_stationary_intervals(AccX: list[float], AccY: list[float], AccZ: list[float]) -> list[float]:
    #各軸の加速度の平均を求める
    AvgAccX = sum(AccX) / len(AccX)
    AvgAccY = sum(AccY) / len(AccY)
    AvgAccZ = sum(AccZ) / len(AccZ)

    #重力加速度の推定値=合成加速度の平均を求める
    AvgResultantAcc = math.sqrt(AvgAccX ** 2 + AvgAccY ** 2 + AvgAccZ ** 2)

    #各時刻の合成加速度を求める
    ResultantAcc = [math.sqrt(x ** 2 + y ** 2 + z ** 2) for x, y, z in zip(AccX, AccY, AccZ)]

    #各時刻の合成加速度から静止区間(重力加速度の推定値に近い値が一定以上以上連続している区間)を除去する
    i=0 #ループ変数
    counter = 0 #静止区間がSTATIONARY_INTERVALS分続いているかをカウントする変数
    while i < len(ResultantAcc):
        if AvgResultantAcc * LOWER_LIMIT < ResultantAcc[i] < AvgResultantAcc * UPPER_LIMIT:   #平均のLOWER_LIMIT倍~UPPER_LIMIT倍の範囲を調べる
            counter += 1    #範囲内ならカウントを増やす
            if counter == STATIONARY_INTERVALS: #カウントがSTATIONARY_INTERVALSに達したらその区間を削除
                del ResultantAcc[i+1-STATIONARY_INTERVALS:i+1]    #スライスでは選択範囲の開始位置startと終了位置stopを[start:stop]のように書く。start <= x < stopの範囲が選択される。start番目の値は含まれるがstop番目の値は含まれない。
                counter = 0
                i -= STATIONARY_INTERVALS   #削除した分インデックスがズレるので補正する
        else:
            counter = 0
        i += 1

    return ResultantAcc  #静止区間を除去した後のリストを返す

In [ ]:
#連続する2サンプルの差分を取る関数
def differences_of_acceleration(ResultantAcc: list[float]) -> list[float]:
    index = 0
    DifferenceAcc = []
    for dif in ResultantAcc[:-1]:
        DifferenceAcc.append(math.fabs(ResultantAcc[index + 1]*100000 - ResultantAcc[index]*100000))
        index += 1

    return DifferenceAcc

In [ ]:
#KLダイバージェンス関数
def KL_divergence(a: list[float], b: list[float]) -> float:
    min_value = min(min(a), min(b)) #a,bの最小値の小さい方
    max_value = max(max(a), max(b)) #a,bの最大値の大きい方

    #a,bのヒストグラムを作成し、同じ数のビンで区切る
    a_hist, _ = np.histogram(a, bins=BINS, range=(min_value, max_value))
    b_hist, _ = np.histogram(b, bins=BINS, range=(min_value, max_value))

    #正規化する(確率分布に変換する、合計を1にする)ために全合計で割る
    a_hist = (a_hist + EPSILON) / a_hist.sum()
    b_hist = (b_hist + EPSILON) / b_hist.sum()

    #KLダイバージェンスの値を返す
    return np.sum([ai * np.log(ai / bi) for ai, bi in zip(a_hist, b_hist)])

In [ ]:
#データセットと周波数を読み込む
path = "data/"  #データセットのあるディレクトリまでのパス
filename_and_Hz = get_filename_and_Hz(path)
filename_and_Hz.sort(reverse=True)
Hz, filename = divide_filename_and_Hz(filename_and_Hz)

AccX, AccY, AccZ = [], [], []
ResultantAcc = []
DifferenceAcc_list = []

for i in filename:
    AccX, AccY, AccZ = get_acceleration(path+i)
    ResultantAcc = remove_stationary_intervals(AccX, AccY, AccZ)
    DifferenceAcc_list.append(differences_of_acceleration(ResultantAcc))

In [ ]:
resultKLD = [[0.0 for j in range(len(filename))] for i in range(len(filename))]  # resultKLDの要素を0.0で初期化
for i in range(len(filename)):
    for j in range(len(filename)):
        resultKLD[i][j] = KL_divergence(DifferenceAcc_list[i], DifferenceAcc_list[j])

In [ ]:
#KLダイバージェンスの結果を格納するデータフレームの定義
df = pd.DataFrame(resultKLD, index=Hz, columns=Hz)